In [1]:
import pandas as pd
import os

#### Add local paths to cluster data

In [2]:
PRE = "/Users/damoncrockett/Desktop/cities_20k/"

In [3]:
DB = "/Users/damoncrockett/Desktop/cities_20k/yahoo/NEW_Clusters/Clusters_Normalized/Normalized_Cluster_Labels/"

In [4]:
data_files = []
for root,dirs,files in os.walk(DB):
    for name in files:
        data_files.append(os.path.join(root,name))

In [5]:
# takes a few minutes to run

for item in data_files:
    tmp = pd.read_csv(item)
    n = len(tmp.index)
    local_path = []
    for i in range(n):
        local_path.append(PRE+tmp.city.loc[i]+"_20k/"+tmp.image_id.loc[i]+".jpg")
    tmp['local_path'] = local_path
    tmp.to_csv(item,index=False)

#### plan histograms

In [6]:
import glob
from PIL import Image

#### now 50x50

In [7]:
num_clusters = 50

In [8]:
thumb_side = 64

In [9]:
DIR = DB+"50/"

In [10]:
df = pd.read_csv(DIR+"food_drinks_meals.csv")
cities = list(set(list(df.city)))

In [11]:
for city in cities:
    outfile = PRE+"viz/normalized_city_food_50/"+city+".png"
    # select rows of df in bin
    tmp = df[df.city==city]
    bin_max = tmp.groupby('cluster_id').size().max()

    # reset index because we'll use the index in a loop
    tmp.reset_index(drop=True,inplace=True)

    # create canvas
    px_w = num_clusters * thumb_side
    px_h = bin_max * thumb_side
    canvas = Image.new('RGB',(px_w,px_h),(50,50,50))
    
    # make bin list
    bins = list(set(list(tmp.cluster_id)))
    for item in bins:
        # define x and y coordinates for pasting
        y_coord = px_h - thumb_side
        x_coord = thumb_side * (item - 1)

        # subset tmp
        subset = tmp[tmp.cluster_id==item]
        subset.reset_index(drop=True,inplace=True)
        
        # loop over rows in subset
        n = len(subset.index)
        for i in range(n):
            thumb = Image.open(subset.local_path.loc[i])
            thumb.thumbnail((thumb_side,thumb_side),Image.ANTIALIAS)
            canvas.paste(thumb,(x_coord,y_coord))
            y_coord = y_coord - thumb_side
    
    canvas.save(outfile)